In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from net import NeuralNetwork
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def fetch_data(ticker, period='10y'):
    data = yf.download(ticker, period=period, interval='1d')
    return data

def preprocess_data(data):
    df = data.copy()
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df.dropna(inplace=True)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    X = df[['MA5', 'MA10', 'RSI']].values  # Shape: (n_samples, 3)
    y = df['Target'].values.reshape(1, -1)  # Shape: (1, n_samples)
    return X.T, y  # X.T: (3, n_samples), y: (1, n_samples)

In [6]:
# Testing data preparation
ticker = 'AAPL'
learning_rate = 0.01
epochs = 10000
batch_size = 64  # Define your mini-batch size

data = fetch_data(ticker)

X, y = preprocess_data(data)  # X.shape: (3, n_samples), y.shape: (1, n_samples)

X_train, X_test, y_train, y_test = train_test_split(
    X.T, y.T, test_size=0.2, random_state=42
)

X_train = X_train.T  # (3, m_train)
X_test = X_test.T    # (3, m_test)
y_train = y_train.T  # (1, m_train)
y_test = y_test.T    # (1, m_test)

print(f"X_train shape: {X_train.shape}")  # Should be (3, m_train)
print(f"y_train shape: {y_train.shape}")  # Should be (1, m_train)
print(f"X_test shape: {X_test.shape}")    # Should be (3, m_test)
print(f"y_test shape: {y_test.shape}")    # Should be (1, m_test)

#print first values of y train first columns
print(y_train[0][:5])


[*********************100%***********************]  1 of 1 completed

X_train shape: (3, 2003)
y_train shape: (1, 2003)
X_test shape: (3, 501)
y_test shape: (1, 501)
[0 1 1 1 0]


In [16]:
ticker = 'AAPL'
learning_rate = 0.1
epochs = 1000
batch_size = 256  # Define your mini-batch size

data = fetch_data(ticker)
if data.empty:
    print("Failed to fetch data. Please check the ticker symbol and try again.")
    exit()

X, y = preprocess_data(data)

# Print initial shapes
print(f"Initial X shape: {X.shape}")  # Expected: (3, m)
print(f"Initial y shape: {y.shape}")  # Expected: (1, m)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X.T, y.T, test_size=0.2, random_state=42
)

# Transpose back to (input_dim, m_train) and (output_dim, m_train)
X_train = X_train.T  # Shape: (3, m_train)
X_test = X_test.T    # Shape: (3, m_test)
y_train = y_train.T  # Shape: (1, m_train)
y_test = y_test.T    # Shape: (1, m_test)

# Print shapes after split
print(f"X_train shape: {X_train.shape}")  # Expected: (3, m_train)
print(f"y_train shape: {y_train.shape}")  # Expected: (1, m_train)")
print(f"X_test shape: {X_test.shape}")    # Expected: (3, m_test)
print(f"y_test shape: {y_test.shape}")    # Expected: (1, m_test)")

scaler = StandardScaler()
# Transpose to fit scaler and then transpose back
X_train = scaler.fit_transform(X_train.T).T  # Shape: (3, m_train)
X_test = scaler.transform(X_test.T).T        # Shape: (3, m_test)

# Print shapes after scaling
print(f"Scaled X_train shape: {X_train.shape}")  # Expected: (3, m_train)
print(f"Scaled X_test shape: {X_test.shape}")    # Expected: (3, m_test)

input_dim = X_train.shape[0]
hidden_dim = 50
output_dim = 1
nn = NeuralNetwork(input_dim, hidden_dim, output_dim)

#initial accuracy, before training
predictions = nn.forward_propagation(X_test)
predictions = (predictions > 0.5).astype(int)
accuracy = np.mean(predictions == y_test) * 100
print(f"Initial Test Accuracy: {accuracy:.2f}%")

# Train using Mini-Batch Gradient Descent
nn.train_mini_batch(
    X_train,
    y_train,
    learning_rate=learning_rate,
    epochs=epochs,
    batch_size=batch_size
)

# Test the network
predictions = nn.forward_propagation(X_test)
predictions = (predictions > 0.5).astype(int)

accuracy = np.mean(predictions == y_test) * 100
print(f"Test Accuracy: {accuracy:.2f}%")

[*********************100%***********************]  1 of 1 completed


Initial X shape: (3, 2504)
Initial y shape: (1, 2504)
X_train shape: (3, 2003)
y_train shape: (1, 2003)
X_test shape: (3, 501)
y_test shape: (1, 501)
Scaled X_train shape: (3, 2003)
Scaled X_test shape: (3, 501)
Initial Test Accuracy: 47.11%
Epoch 0, Loss: 0.250661
Epoch 100, Loss: 0.249252
Epoch 200, Loss: 0.249233
Epoch 300, Loss: 0.249217
Epoch 400, Loss: 0.249201
Epoch 500, Loss: 0.249196
Epoch 600, Loss: 0.249172
Epoch 700, Loss: 0.249154
Epoch 800, Loss: 0.249138
Epoch 900, Loss: 0.249123
Test Accuracy: 52.50%
